### Importação das bibliotecas

In [1]:
import pandas as pd
import numpy as np

import sqlalchemy
from urllib.parse import quote_plus
from sqlalchemy import create_engine, text
from sqlalchemy.engine.base import Connection
from sqlalchemy.engine import URL

### Conexão com o Banco de Dados

In [16]:

def conn_bd():

    # pyodbc stuff for MS SQL Server Express
    driver='{ODBC Driver 17 for SQL Server}'
    server='DESKTOP-LG9U8DH'
    database='Raizen'
    trusted_connection='yes'

    # pyodbc connection string
    connection_string = f'DRIVER={driver};SERVER={server};\
                          DATABASE={database};\
                          ;\ TRUSTED_CONNECTION={trusted_connection}'
    
    connection_string += f'DATABASE={database};'
    connection_string += f'TRUSTED_CONNECTION={trusted_connection}'



    try:
    # create sqlalchemy engine connection URL
        connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
        engine = sqlalchemy.create_engine(connection_url)
        print("Conexao realizada!")
    except Exception as error:
        error = str(error)
        print("Conexao não realizada! " + error)


    
    return engine

### Leitura dos dados 

In [2]:
caminho = r'C:\Users\BlueShift\Desktop\Teste Raizen\dados\vendas-combustiveis-m3-anp-ate-jan-2023.xlsx'

In [3]:
planilha = pd.read_excel(caminho, skiprows=2, sheet_name=None)

In [4]:
planilha

{'2000':          COMBUSTÍVEL   ANO               REGIÃO              ESTADO UNIDADE  \
 0    GASOLINA C (m3)  2000         REGIÃO NORTE            RONDÔNIA      m3   
 1    GASOLINA C (m3)  2000         REGIÃO NORTE                ACRE      m3   
 2    GASOLINA C (m3)  2000         REGIÃO NORTE            AMAZONAS      m3   
 3    GASOLINA C (m3)  2000         REGIÃO NORTE             RORAIMA      m3   
 4    GASOLINA C (m3)  2000         REGIÃO NORTE                PARÁ      m3   
 ..               ...   ...                  ...                 ...     ...   
 211         GLP (m3)  2000           REGIÃO SUL   RIO GRANDE DO SUL      m3   
 212         GLP (m3)  2000  REGIÃO CENTRO-OESTE  MATO GROSSO DO SUL      m3   
 213         GLP (m3)  2000  REGIÃO CENTRO-OESTE         MATO GROSSO      m3   
 214         GLP (m3)  2000  REGIÃO CENTRO-OESTE               GOIÁS      m3   
 215         GLP (m3)  2000  REGIÃO CENTRO-OESTE    DISTRITO FEDERAL      m3   
 
               Jan           F

In [5]:
planilha.keys()

dict_keys(['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', 'Plan1'])

### Concatenando todos os dataframes referentes aos anos

In [6]:
df = pd.DataFrame()

In [7]:
for i in planilha:
    if i != 'Plan1':
        df = pd.concat([df, planilha[i]])

In [8]:
df.shape

(4968, 18)

In [9]:
df.sample(15)

,COMBUSTÍVEL,ANO,REGIÃO,ESTADO,UNIDADE,Jan,Fev,Mar,Abr,Mai,Jun,Jul,Ago,Set,Out,Nov,Dez,TOTAL
10,GLP (m3),2002,REGIÃO SUDESTE,MINAS GERAIS,m3,109518.418182,108796.800000,129786.134545,116698.221818,124600.130909,112797.714545,130094.123636,116912.229091,114691.809091,120470.672727,114015.529091,113624.327273,1.412006e+06
187,GASOLINA DE AVIAÇÃO (m3),2005,REGIÃO NORTE,ACRE,m3,29.712000,29.637000,59.673000,33.759000,59.856000,89.820000,30.276000,59.892000,60.000000,64.544000,29.748000,59.712000,6.066290e+02
146,QUEROSENE ILUMINANTE (m3),2009,REGIÃO NORDESTE,BAHIA,m3,13.000000,59.000000,24.600000,19.200000,34.200000,26.400000,8.800000,14.000000,26.800000,24.000000,27.000000,3.000000,2.800000e+02
26,GLP (m3),2017,REGIÃO NORTE,RONDÔNIA,m3,7309.969203,7126.903986,8737.367299,7306.280797,8024.963768,8153.318631,7610.355072,8204.772245,7602.467848,7639.989032,7520.853027,7869.111740,9.310635e+04
32,ETANOL HIDRATADO (m3),2008,REGIÃO SUL,SANTA CATARINA,m3,27860.291194,26718.557280,27078.095001,28702.004271,28447.593445,27400.358771,30450.606838,30201.260952,33721.934041,35652.982850,35757.722037,44167.070321,3.761585e+05
96,ÓLEO DIESEL (m3),2011,REGIÃO NORDESTE,PARAÍBA,m3,33687.050530,32823.897377,35569.094027,32137.084473,33470.753795,33797.857342,34251.976353,37612.283468,38303.466323,38613.751888,38328.286695,40426.066607,4.290216e+05
28,ETANOL HIDRATADO (m3),2005,REGIÃO CENTRO-OESTE,GOIÁS,m3,12587.885000,11664.027000,12774.707000,9560.079000,9023.588000,8660.283000,9848.802000,10635.821000,13449.336000,13732.496000,15499.752000,21939.107000,1.493759e+05
106,ÓLEO DIESEL (m3),2014,REGIÃO NORTE,ACRE,m3,12472.625000,12053.827000,10462.000000,11876.001000,12915.000000,13873.080000,15840.000000,14816.000000,16702.500000,16864.000000,13622.000000,15493.156000,1.669902e+05
197,GASOLINA C (m3),2002,REGIÃO SUDESTE,RIO DE JANEIRO,m3,148357.094000,146240.162000,163590.049000,171545.224000,160282.966000,151016.755000,165306.179000,170212.081000,160583.431000,192342.443000,163395.830000,179062.033000,1.971934e+06
151,QUEROSENE ILUMINANTE (m3),2004,REGIÃO NORDESTE,RIO GRANDE DO NORTE,m3,70.000000,70.000000,35.000000,35.000000,65.000000,70.000000,35.000000,45.000000,65.000000,40.000000,55.000000,62.000000,6.470000e+02


In [10]:
df = df.sort_values(by=['ESTADO', 'ANO'], ascending=True)

In [11]:
df.head(4)

,COMBUSTÍVEL,ANO,REGIÃO,ESTADO,UNIDADE,Jan,Fev,Mar,Abr,Mai,Jun,Jul,Ago,Set,Out,Nov,Dez,TOTAL
1,GASOLINA C (m3),2000,REGIÃO NORTE,ACRE,m3,3065.758,3495.290,2946.930,3023.920,3206.930,3612.580,3264.460,3835.740,3676.571,3225.610,3289.718,3358.346,40001.853
28,GASOLINA DE AVIAÇÃO (m3),2000,REGIÃO NORTE,ACRE,m3,45.450,30.300,45.450,30.300,30.300,60.600,30.160,60.600,77.687,46.455,3.525,2.984,463.811
55,QUEROSENE ILUMINANTE (m3),2000,REGIÃO NORTE,ACRE,m3,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
82,QUEROSENE DE AVIAÇÃO (m3),2000,REGIÃO NORTE,ACRE,m3,1154.561,1250.719,812.783,702.337,964.217,937.929,1119.195,1098.881,1046.794,1061.304,1417.661,1404.932,12971.313


### Exploração dos dados

In [15]:
df.columns

Index(['COMBUSTÍVEL', 'ANO', 'REGIÃO', 'ESTADO', 'UNIDADE', 'Jan', 'Fev',
       'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez',
       'TOTAL'],
      dtype='object')

In [12]:
df[df['ESTADO'] == 'ACRE'].sum(axis=0, numeric_only=True)

ANO      3.700240e+05
Jan      5.802779e+05
Fev      5.082649e+05
Mar      5.618877e+05
Abr      5.413359e+05
Mai      5.757422e+05
Jun      5.757038e+05
Jul      6.306767e+05
Ago      6.517391e+05
Set      6.500501e+05
Out      6.667208e+05
Nov      6.059734e+05
Dez      6.126823e+05
TOTAL    7.161055e+06
dtype: float64

In [13]:
estado_lista = []
estado_lista = df.ESTADO.unique()
estado_lista

array(['ACRE', 'ALAGOAS', 'AMAPÁ', 'AMAZONAS', 'BAHIA', 'CEARÁ',
       'DISTRITO FEDERAL', 'ESPÍRITO SANTO', 'GOIÁS', 'MARANHÃO',
       'MATO GROSSO', 'MATO GROSSO DO SUL', 'MINAS GERAIS', 'PARANÁ',
       'PARAÍBA', 'PARÁ', 'PERNAMBUCO', 'PIAUÍ', 'RIO DE JANEIRO',
       'RIO GRANDE DO NORTE', 'RIO GRANDE DO SUL', 'RONDÔNIA', 'RORAIMA',
       'SANTA CATARINA', 'SERGIPE', 'SÃO PAULO', 'TOCANTINS'],
      dtype=object)

In [14]:
for i in estado_lista:
    print(i)
    print(f"{df[df['ESTADO'] == i].sum(axis=0, numeric_only=True)}")
    print('='*25)

ACRE
ANO      3.700240e+05
Jan      5.802779e+05
Fev      5.082649e+05
Mar      5.618877e+05
Abr      5.413359e+05
Mai      5.757422e+05
Jun      5.757038e+05
Jul      6.306767e+05
Ago      6.517391e+05
Set      6.500501e+05
Out      6.667208e+05
Nov      6.059734e+05
Dez      6.126823e+05
TOTAL    7.161055e+06
dtype: float64
ALAGOAS
ANO      3.700240e+05
Jan      1.887780e+06
Fev      1.663658e+06
Mar      1.716722e+06
Abr      1.561802e+06
Mai      1.548655e+06
Jun      1.500726e+06
Jul      1.597942e+06
Ago      1.658633e+06
Set      1.717022e+06
Out      1.906874e+06
Nov      1.881022e+06
Dez      2.039284e+06
TOTAL    2.068012e+07
dtype: float64
AMAPÁ
ANO      3.700240e+05
Jan      6.735179e+05
Fev      6.064683e+05
Mar      6.524638e+05
Abr      6.120806e+05
Mai      6.442839e+05
Jun      6.561043e+05
Jul      6.879383e+05
Ago      7.450048e+05
Set      7.333924e+05
Out      7.948567e+05
Nov      8.091161e+05
Dez      8.210424e+05
TOTAL    8.436269e+06
dtype: float64
AMAZONAS
ANO

### Envio dos dados para o Banco de dados no schema Stage

In [17]:
engine = conn_bd()

df.to_sql(name= 'derivados_petroleo',con=engine, if_exists='append', index=False, schema='Stage')

Conexao realizada!


96